<a href="https://colab.research.google.com/github/MinseoYoo/bitamin_nlp_2024_2/blob/main/%5B%EB%AF%BC%EC%84%9C%5D%20T5%20%EB%AA%A8%EB%8D%B8%EC%8B%9C%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/비타민 14기/학기 NLP/output_1000.csv')

In [4]:
# 새로운 데이터를 저장할 리스트 초기화
all_data = []

# 각 행에 대해 처리
for i in range(len(df)):
    subject = df['subject'][i]
    text = df['text'][i]
    lines = text.split('\n')

    for line in lines:
        if " : " in line:
            dialogue = line.split(" : ", 1)[1].strip()
            all_data.append({"dialogue": dialogue, "subject": subject})

df_new = pd.DataFrame(all_data)

In [5]:
df_new = df_new[:100]

In [6]:
df_new

,dialogue,subject
0,인도는 폭우가 와서 난리라네,타 국가 이슈
1,요새 그냥 지구가 난리다 ㅠㅠ,타 국가 이슈
2,인도는 요새 계속 난리네 ?,타 국가 이슈
3,사람이 벌써 30명이 죽었대!,타 국가 이슈
4,우리나라도 그렇고 하하,타 국가 이슈
...,...,...
95,난 비싼 거 안 씀 어짜피 실력임,상거래 전반
96,이태곤 아저씨 집도 꼭 봐봐...,상거래 전반
97,오 **도 실력파인데,상거래 전반
98,놉 30마넌짜리로 잡는 거임 키키,상거래 전반


In [7]:
def preprocess_data(df):
    subject = df['subject']
    dialogue = df['dialogue'].replace("\n", " ")
    data_dict = {'topic': subject, 'dialogue': dialogue}
    return data_dict

data_dict = preprocess_data(df_new)

In [8]:
model_name = 'KETI-AIR/ke-t5-base'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

dataset = Dataset.from_dict(data_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [9]:
# 입력과 출력 텍스트를 처리하는 함수 정의
def preprocess_function(examples):
    inputs = [f"주제: {topic}" for topic in examples['topic']]
    outputs = [f"{dialogue}" for dialogue in examples['dialogue']]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(outputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt").input_ids

    model_inputs["labels"] = labels
    return model_inputs

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",          # 출력 디렉토리
    evaluation_strategy="epoch",     # 평가 주기
    learning_rate=5e-5,              # 학습률
    per_device_train_batch_size=8,   # 배치 크기
    per_device_eval_batch_size=8,    # 평가 배치 크기
    num_train_epochs=10,              # 에폭 수
    weight_decay=0.01,               # 가중치 감쇠
    save_total_limit=2,              # 저장할 체크포인트 수
    save_steps=500,                  # 체크포인트 저장 주기
    report_to="none",
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# 모델 훈련
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,89.422142
2,No log,79.444664
3,No log,72.503754
4,No log,66.502213
5,No log,61.568531
6,No log,57.349136
7,No log,54.513073
8,No log,52.509239
9,No log,51.330902
10,No log,50.901722


TrainOutput(global_step=130, training_loss=84.01483623798077, metrics={'train_runtime': 103.6203, 'train_samples_per_second': 9.651, 'train_steps_per_second': 1.255, 'total_flos': 608957890560000.0, 'train_loss': 84.01483623798077, 'epoch': 10.0})